In [1]:
import os

import numpy as np
import pandas as pd

In [2]:
dir_csv = '../data/bike_rental_history/raw'
list_csv = os.listdir(dir_csv)

list_df_bike = []
for csv in list_csv:
  df_tmp = pd.read_csv(f'{dir_csv}/{csv}', encoding='cp949')
  list_df_bike.append(df_tmp)

df_bike_rental = pd.concat(list_df_bike, ignore_index=True)

# 대여대여소, 반납대여소 id가 없는 대여기록은 삭제
df_bike_rental = df_bike_rental.loc[np.logical_not(pd.isna(df_bike_rental.loc[:,'대여대여소ID']))]
df_bike_rental = df_bike_rental.loc[np.logical_not(pd.isna(df_bike_rental.loc[:,'반납대여소ID']))]
df_bike_rental = df_bike_rental.loc[np.logical_not(df_bike_rental.loc[:,'반납대여소ID'].str.contains('N'))]

# 대여시간 자료형 변환
df_bike_rental.loc[:,'대여일시'] = pd.to_datetime(df_bike_rental.loc[:,'대여일시'])

C:\Users\seung\AppData\Local\Temp\ipykernel_16812\57293592.py:17: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  df_bike_rental.loc[:,'대여일시'] = pd.to_datetime(df_bike_rental.loc[:,'대여일시'])


In [19]:
df_bike_rental_weekday = df_bike_rental.loc[df_bike_rental.loc[:,'대여일시'].dt.dayofweek <5]
df_bike_rental_weekday = df_bike_rental_weekday.loc[df_bike_rental_weekday.loc[:,'대여대여소ID'] != df_bike_rental_weekday.loc[:,'반납대여소ID']]
df_bike_rental_weekday = df_bike_rental_weekday.sample(frac=1/df_bike_rental_weekday.loc[:,'대여일시'].dt.date.nunique())
df_bike_rental_weekday.loc[:,'route'] = df_bike_rental_weekday.loc[:,['대여대여소ID','반납대여소ID']].apply(lambda x: '~'.join(x), axis=1)

In [20]:
df_bike_rental_weekday_count = df_bike_rental_weekday.groupby('route', as_index=False).count().loc[:,['route','자전거번호']].rename(columns={'자전거번호':'count'})
df_bike_rental_weekday_count = df_bike_rental_weekday_count.loc[df_bike_rental_weekday_count.loc[:,'count']>1]

df_bike_rental_weekday = df_bike_rental_weekday.loc[df_bike_rental_weekday.loc[:,'route'].isin(df_bike_rental_weekday_count.loc[:,'route'])]

In [21]:
df_bike_rental_weekday.loc[:,'route'].nunique()

21425

In [22]:
df_bike_rental_weekday.to_csv('../data/bike_rental_history/processed/2022_weekday_sample.csv', index=False)